# Landsat Collection 2 downloader
michael.brechbuehler@eawag.ch

This notebook uses the [landsatxplore](https://github.com/yannforget/landsatxplore) python package to download Landsat Collection 2 scenes directly from USGS.

### Setup environment
To setup a new environment (using conda):
> conda create --name landsatxplore -c conda-forge python=3 notebook nb_conda_kernels geopandas hvplot cartopy geoviews

To setup a new environment (using conda and environment.yml):
> conda env create -n landsatxplore --file environment.yml

### Imports

In [211]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import shapely
import hvplot.pandas
from tqdm import tqdm
from pathlib import Path
import tarfile
from contextlib import closing

# import landsatxplore
import landsatxplore
from landsatxplore.earthexplorer import EarthExplorer
from landsatxplore.api import API

## 1. Search query

### USGS EROS credentials
To allow queries and file downloads from the USGS EROS webserver a free account is necessary (https://ers.cr.usgs.gov/register)

In [218]:
# Your USGS  credentials
username = ".."
password = ".."

Now we can set the desired search parameters for our Landsat query. We also have to define which Landsat collection to use.

| Dataset Name | Dataset ID |
| :-- | -- |
| Landsat 5 TM Collection 2 Level 1 | `landsat_tm_c2_l1` |
| Landsat 5 TM Collection 2 Level 2 | `landsat_tm_c2_l2` |
| Landsat 7 ETM+ Collection 2 Level 1 | `landsat_etm_c2_l1` |
| Landsat 7 ETM+ Collection 2 Level 2 | `landsat_etm_c2_l2` |
| Landsat 8 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 8 Collection 2 Level 2 | `landsat_ot_c2_l2` |
| Landsat 9 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 9 Collection 2 Level 2 | `landsat_ot_c2_l2` |

### Search parameters

In [219]:
# Search parameters
bbox = (6.117096, 46.189198, 6.938324 ,46.544463) # (lonmin, latmin, lonmax, latmax)

search_params = {
    'dataset': 'landsat_ot_c2_l1',
    'bbox': bbox, 
    'start_date': '2023-01-01',
    'end_date': '2023-12-31',
    'months': [1,2,3,4,5,6,7,8,9,10,11,12],
    'max_cloud_cover': 100,
    'max_results': 1000, # Defaults to 100 if not specified
}

### Plot ROI

In [220]:
# Plot bounding box on map
bbox_poly = shapely.geometry.box(*bbox, ccw=True)
df_bbox = gpd.GeoDataFrame([], geometry=[bbox_poly])

plot_bbox = df_bbox.hvplot(title='ROI bounding box', geo=True, 
                           fill_alpha=0.2, color='red', line_color='red', 
                           #tiles='CartoLight'
                           tiles='OSM', label='ROI'
                          )

plot_bbox

:Overlay
   .WMTS.I       :WMTS   [Longitude,Latitude]
   .Polygons.ROI :Polygons   [Longitude,Latitude]

### Start search query

In [221]:
# initialize a new API instance
api = API(username, password)

# search for Landsat 8-9 L2C2 scenes
scenes = api.search(**search_params)

# logout
api.logout()

# create a GeoDataFrame from the returned metadata
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date', 'spatial_coverage']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes['satellite'] = df_scenes.satellite.apply(lambda x: f'Landsat-{x}')
df_scenes['tile'] = df_scenes.display_id.apply(lambda x: x.split('_')[2][:3]+'/'+x.split('_')[2][3:])
gdf_scenes = gpd.GeoDataFrame(df_scenes.drop(columns='spatial_coverage'), geometry=df_scenes.spatial_coverage, crs='EPSG:4326')

gdf_scenes

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date,tile,geometry
0,LC09_L1TP_196027_20231107_20231107_02_T2,196,27,Landsat-9,90,2023-11-07,196/027,"POLYGON ((4.87442 46.80431, 7.29193 46.37052, ..."
1,LC09_L1TP_196028_20231107_20231107_02_T1,196,28,Landsat-9,76,2023-11-07,196/028,"POLYGON ((4.37992 45.38048, 6.73762 44.95365, ..."
2,LC09_L1TP_195028_20231031_20231031_02_T1,195,28,Landsat-9,79,2023-10-31,195/028,"POLYGON ((5.92512 45.38061, 8.28257 44.95382, ..."
3,LC08_L1GT_196027_20231030_20231030_02_RT,196,27,Landsat-8,100,2023-10-30,196/027,"POLYGON ((4.86389 46.80551, 7.28102 46.36914, ..."
4,LC08_L1GT_196028_20231030_20231030_02_RT,196,28,Landsat-8,100,2023-10-30,196/028,"POLYGON ((4.36896 45.38172, 6.72631 44.95233, ..."
...,...,...,...,...,...,...,...,...
113,LC08_L1GT_196027_20230115_20230131_02_T2,196,27,Landsat-8,96,2023-01-15,196/027,"POLYGON ((4.87241 46.80517, 7.28974 46.36939, ..."
114,LC08_L1GT_196028_20230115_20230131_02_T2,196,28,Landsat-8,100,2023-01-15,196/028,"POLYGON ((4.37817 45.38130, 6.73570 44.95248, ..."
115,LC08_L1GT_195028_20230108_20230124_02_T2,195,28,Landsat-8,100,2023-01-08,195/028,"POLYGON ((5.91210 45.38145, 8.26950 44.95259, ..."
116,LC09_L1TP_196027_20230107_20230314_02_T1,196,27,Landsat-9,11,2023-01-07,196/027,"POLYGON ((4.84817 46.80420, 7.26584 46.37081, ..."


### Plot available tiles over ROI

In [222]:
gdf_tiles = gdf_scenes.groupby('tile').first().reset_index()[['tile', 'wrs_path', 'wrs_row', 'geometry']].set_crs('EPSG:4326')
gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)

plot_labels = gdf_labels.to_crs("EPSG:3857").assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y).hvplot.labels(text="tile", x="x", y="y", text_color='white')

plot_tiles = gdf_tiles.hvplot(title='Available tiles over ROI', geo=True,
                              fill_alpha=0.3, line_color='blue',
                              height=800, tiles='OSM', label='Landsat tile'
                             ) * plot_bbox * plot_labels

plot_tiles

C:\Users\Michael\AppData\Local\Temp\ipykernel_23392\3331222002.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)


:Overlay
   .WMTS.I                :WMTS   [Longitude,Latitude]
   .Polygons.Landsat_tile :Polygons   [Longitude,Latitude]
   .WMTS.II               :WMTS   [Longitude,Latitude]
   .Polygons.ROI          :Polygons   [Longitude,Latitude]
   .Labels.I              :Labels   [x,y]   (tile)

### Plot all available scenes

In [226]:
plot_scenes = df_scenes.hvplot.line(title=f'Available Landsat C2 scenes over ROI\n{search_params["start_date"]} to {search_params["end_date"]}, n={df_scenes.shape[0]}',
    x='acquisition_date', y='cloud_cover', color='grey', grid=True) *\
                df_scenes.hvplot.scatter(x='acquisition_date', y='cloud_cover', c='satellite',
                                       hover_cols=['tile', 'acquisition_date'],
                                       xlabel='Acq. date', ylabel='Cloud cover (%)', marker='o')
plot_scenes

:Overlay
   .Curve.I   :Curve   [acquisition_date]   (cloud_cover)
   .Scatter.I :Scatter   [acquisition_date]   (cloud_cover,satellite,tile)

## 2. Download scenes
### Filter and select scene IDs

In [227]:
# Select scene ids to download from dataframe
#ids = df_scenes.display_id.values # all
df_scenes_filt = df_scenes.loc[(df_scenes.cloud_cover < 2) & (df_scenes.tile=='196/028')] # filter by cloudcover and tile
ids = df_scenes_filt.display_id.values
df_scenes_filt

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date,spatial_coverage,tile
34,LC08_L1TP_196028_20230811_20230818_02_T1,196,28,Landsat-8,1,2023-08-11,"POLYGON ((4.39496 45.38193, 6.75219 44.95209, ...",196/028
52,LC08_L1TP_196028_20230624_20230630_02_T1,196,28,Landsat-8,1,2023-06-24,"POLYGON ((4.39833 45.382, 6.7553 44.95215, 7.4...",196/028


### Download .tar files

In [228]:
# set output directory
path_output_dir = Path(r'E:/landsat_c2l2')

# initialize the API
ee = EarthExplorer(username, password)

# download the scenes
for id in tqdm(ids, total=len(ids), desc="Total download progress", position=-1):
    path_output_file = path_output_dir.joinpath(id+'.tar')
    try:
        ee.download(id, output_dir=path_output_dir)
        print('{} successful'.format(id))

  # aditional error handling
    except:
        if path_output_file.exists():
            print(f'{id} error but file exists')
        else:
            print(f'{id} error')

ee.logout()


100%|█████████████████████████████████████████████████████████████████████████████| 1.18G/1.18G [00:42<00:00, 29.8MB/s]

Total download progress:  50%|█████████████████████████████▌                             | 1/2 [00:48<00:48, 48.55s/it]

LC08_L1TP_196028_20230811_20230818_02_T1 successful
Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|█████████████████████████████████████████████████████████████████████████████| 1.19G/1.19G [00:37<00:00, 33.9MB/s]

Total download progress: 100%|███████████████████████████████████████████████████████████| 2/2 [02:16<00:00, 68.45s/it]


LC08_L1TP_196028_20230624_20230630_02_T1 successful


### Extract .tar files

In [229]:
# Extract .tar files into folders
paths_output = list(path_output_dir.glob('*.tar'))

for path in tqdm(paths_output, desc="Total extraction progress"):
    with closing(tarfile.open(path)) as fl:
        path_output_folder = path_output_dir.joinpath(path.name.split('.')[0])
        path_output_folder.mkdir(parents=True, exist_ok=True)
        fl.extractall(path_output_folder)
    path.unlink()

Total extraction progress: 100%|█████████████████████████████████████████████████████████| 2/2 [00:29<00:00, 14.63s/it]
